### Building Chatbot with Multiple Tools Using langGraph

1. Aim
- Create a chatbot with tool capabilities from arxivm wikipedia search and some function.

In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper


In [2]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [3]:
arxiv.invoke("What is the latest research on LangChain?")

'Published: 2016-02-24\nTitle: Concentration of research funding leads to decreasing marginal returns\nAuthors: Philippe Mongeon, Christine Brodeur, Catherine Beaudry, Vincent Lariviere\nSummary: In most countries, basic research is supported by research councils that select, after peer review, the individuals or teams that are to receive funding. Unfortunately, the number of grants these research councils can allocate is not infinite and, in most cases, a minority of the researchers receive the maj'

In [4]:
api_wrapper_wikipedia=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=500)
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)
wikipedia.name

'wikipedia'

In [5]:
wikipedia.invoke("What is the latest research on LangChain?")

'Page: Wednesday (TV series)\nSummary: Wednesday is an American supernatural mystery comedy television series based on the character Wednesday Addams by Charles Addams. Created by Alfred Gough and Miles Millar, it stars Jenna Ortega as the titular character, with Luis Guzmán and Catherine Zeta-Jones portraying her parents in the series. Four out of the eight episodes of the first season were directed by Tim Burton, who also was executive producer. The first season revolves around Wednesday Addams,'

In [7]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [8]:
## Tavily search
from langchain_community.tools.tavily_search import TavilySearchResults
tavily=TavilySearchResults(max_results=2)
tavily.invoke("Provide me the latest AI news")

C:\Users\Hello\AppData\Local\Temp\ipykernel_15088\3885505669.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily=TavilySearchResults(max_results=2)


[{'title': 'AI News | Latest News | Insights Powering AI-Driven Business Growth',
  'url': 'https://www.artificialintelligence-news.com/',
  'content': 'AI News delivers the latest updates in artificial intelligence, machine learning, deep learning, enterprise AI, and emerging tech worldwide.',
  'score': 0.99837697},
 {'title': 'The Latest AI News and AI Breakthroughs that Matter Most: 2025',
  'url': 'https://www.crescendo.ai/news/latest-ai-news-and-updates',
  'content': 'Summary: Xiaomi has announced a next-gen AI voice model optimized for in-car and smart home experiences. The model features faster response times, offline',
  'score': 0.9891816}]

In [9]:
## Combine all tools
tools=[arxiv,wikipedia,tavily]
## Initialize the LLM Modell
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API_KEY"))
llm_with_tools=llm.bind_tools(tools)

In [ ]:
from pprint import pprint
from langchain_core.messages import HumanMessage,AIMessage
llm_with_tools.invoke([HumanMessage(content="What is the latest AI news?")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'psj0c64g1', 'function': {'arguments': '{"query":"latest AI news"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 554, 'total_tokens': 574, 'completion_time': 0.023358951, 'completion_tokens_details': None, 'prompt_time': 0.030823186, 'prompt_tokens_details': None, 'queue_time': 0.021796118, 'total_time': 0.054182137}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6b5c123dd9', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--077c224e-cd14-492e-8cb1-73ce7fd0a05b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'latest AI news'}, 'id': 'psj0c64g1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 554, 'output_tokens': 20, 'total_tokens': 574})

In [11]:
llm_with_tools.invoke([HumanMessage(content="What is the latest AI news?")]).tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'latest AI news'},
  'id': 'enfswm14x',
  'type': 'tool_call'}]